# [Advent of Code - 2017](https://adventofcode.com/2017)

Advent of code is a puzzle solving website, two puzzles released for each day of advent (Dec 1st - Dec 25th). If previous years are anything to go by the cover a large variety of algorithms and are generally quite fun!

Each year the puzzels are built around a central theme, with this year's theme being that we have been "digitized" into a computer, and must solve various problems from inside the machine.

# Day 0

This portion contains various common pieces of code that'll be used on multiple days.

In [1]:
from itertools import cycle, count
import os
import re

        
def Input(day):
    """Fetch the data input from disk."""
    filename = os.path.join('../data/advent2017/input{}.txt'.format(day))
    return open(filename)


def neighbours4(x, y):
    return (x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)


def neighbours8(x, y):
    return (
        (x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1),
        (x - 1, y - 1), (x - 1, y + 1), (x + 1, y - 1), (x + 1, y + 1)
    )


def manhattan_distance(point1, point2):
    return abs(point2[0] - point1[0]) + abs(point2[1] - point1[1])

# [Day 1: Inverse Captcha](https://adventofcode.com/2017/day/1)

We're greeted by a door, and must proove that we are _not_ human to continue. The first puzzle has us performing a captha that "only a computer" can solve. We're required to sum digits in a list where each digit matches the one immediately following it, wrapping to the start if we overflow.

In [2]:
def sum_if_match(nums, jump_distance):
    total = 0
    for index, n in enumerate(nums):
        next_n = nums[(index + jump_distance) % len(nums)] 
        if n == next_n:
            total += n
    return total

def sum_consecutive(data):
    nums = list(map(int, data))
    jump_distance = 1
    return sum_if_match(nums, jump_distance)


assert sum_consecutive('1122') == 3
assert sum_consecutive('1111') == 4
assert sum_consecutive('1234') == 0
assert sum_consecutive('91212129') == 9

sum_consecutive(Input(1).read().strip())

1182

For the second portion, we're again summing digits, but now only if we match the digit exactly _half the list_ away. At this point we can modify the initial code and provide a jump distance.

In [3]:
def sum_half(data):
    nums = list(map(int, data))
    jump_distance = len(nums) // 2
    return sum_if_match(nums, jump_distance)
    

assert sum_half('1212') == 6
assert sum_half('1221') == 0
assert sum_half('123425') == 4
assert sum_half('123123') == 12
assert sum_half('12131415') == 4

sum_half(Input(1).read().strip())

1152

# [Day 2: Corruption Checksum](https://adventofcode.com/2017/day/2)

Here we're required to perform some data anaylsis on a spreadsheet calculating a checksum of each row by finding the difference of the max and min values.

In [4]:
def parse_input(data):
    as_ints = []
    for row in data.split('\n'):
        if not row:
            break
        nums = list(map(int, re.findall('\d+', row)))
        as_ints.append(nums)
    return as_ints


data = parse_input(Input(2).read())

sum((max(row) - min(row) for row in data))

42378

The second portion requires us to find pairs of number in each row that are evenly divisible, and summing the result of their division. As we don't know in which order the pair will appear, I sort each row when meeting it.

In [5]:
def sum_even_div(data):
    total = 0
    for nums in data:
        nums = sorted(nums)
        for i, x in enumerate(nums[:-1]):
            for y in nums[i + 1:]:
                if y % x == 0:
                    total += y // x
                    break
    return total


sum_even_div(data)

246

# [Day 3: Spiral Memory](https://adventofcode.com/2017/day/3)

We need to find the coordinate of an number when it's displayed in a spiral format. I.e.

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```

I started this problem by working out an equation to find the location of the Nth element, without building the rest of the grid. While this worked well for the first part of the problem, the second part requires us to build a spiral grid anyway! (It's much smaller, but still.)

In [9]:
def find_coorindates(N):
    """Find the coordinates of N in a spiral matrix.
    
    We can find the shell that the number occurs in by
    finding the upper limit for each shell and 
    """
    # First find the shell that the number appears in
    # number of elements in each shell is 4*(n-1)
    if N == 1:
        return 0, 0

    shell = 1
    shell_size = 1
    while N > shell_size ** 2:
        shell += 1
        shell_size += 2

    shell_end = shell_size ** 2
    shell_start = (shell_size - 2) ** 2
    elms_in_shell = shell_end - shell_start
    position_in_shell = N - shell_start
    
    side_length = elms_in_shell // 4
    half_side = side_length // 2
    
    side = position_in_shell / elms_in_shell
    
    if side <= 0.25:
        # right
        x = half_side
        y = (position_in_shell % side_length) - half_side
    elif side <= 0.5:
        # top
        x = (position_in_shell % side_length) - half_side
        y = half_side
    elif side <= 0.75:
        # left
        x = -half_side
        y = (position_in_shell % side_length) - half_side
    else:
        # bottom
        x = (position_in_shell % side_length) - half_side
        y = -half_side
    return (x, y)
    
    
def carry_distance(N):
    x, y = find_coorindates(N)
    return manhattan_distance((0, 0), (x, y))


data = 312051
assert carry_distance(1) == 0
assert carry_distance(12) == 3
assert carry_distance(23) == 2
assert carry_distance(1024) == 31
carry_distance(data)

430

The second part requires us to perform a "stress test" to populate a spiral grid with values the sum of all adjacent cells in the grid. As we don't know the upper bound (and I don't know how to initialize an infinite grid that can be referenced arbitarily..) I went with a dictionay to store point references and their values. 

By combining two infinite generators that cycle through the directions we turn and the distances we need to travel, we build an a third infinite generator that contains all the steps we'll take.

In [10]:
from itertools import cycle, count

def spiral_distances():
    """"Yields 1, 1, 2, 2, 3, 3, ...
    
    As the spiral wraps around itself, we increase
    the distance we travel by 1 every two distances.
    
    This is because every 2 distances we're moving in
    the opposite direction, so to we increase the
    distance to ensure we can move past the movement
    we're now opposing.
    """
    for distance in count(1):
        for _ in (0, 1):
            yield distance

            
def directions():
    """Yields R, U, L, D, R, U, L, D, ..."""
    up = (0, -1)
    down = (0, 1)
    left = (-1, 0)
    right = (1, 0)
    return cycle((right, up, left, down))


def spiral_movements():
    for distance, direction in zip(spiral_distances(), directions()):
        for _ in range(distance):
            yield direction


def stress_test(max_val):
    grid = {}
    x, y = 0, 0
    grid[(x, y)] = 1
    for direction in spiral_movements():
        dx, dy = direction
        x += dx
        y += dy
        val = sum(
            grid.get(neighbour, 0)
            for neighbour in neighbours8(x, y)
        )

        grid[(x, y)] = val

        if val > max_val:
            return val
stress_test(data)
    

312453